### Loads data, computes persistence diagrams. 7/6/18

The goal here is to first access our training2017.zip folder and compute persistence diagrams. The code here first
1. Stores 4 organized lists of length $k$, (we took $k$ to be 200 in our analysis) where each list is organized by classes of ECG data. The four classes are normal, atrial fibrillation, random/noise (denoted random here on out) and other. 

2. The function reconstruct_persist systematically computes persistence diagrams in a form that R can accept. 

3. Eight_Hundred_Persistence_Test takes in a folder containing the ECG graphs. It assumes you have them organized in four neat folders called Normal, AF, Random, and Other. It's called 800 because we set $k = 200.$ It then stores these persistence diagrams into csvs, although you can just use this output and plug it into reconstruct_persist to reformat the data so that R can take it in. 

4. TXT_to_ARRAY assumes also takes in a folder and it assumes you have data written in .txt form in four neat folders, again called Normal, AF, Random, and Other (right now we only have 3, but that can be added later). The reason why it takes txt files is because that is the output of another function, which cleans up the data and removes noise.

5. Persistence_Lists takes in clean data and returns a list where each element is the persistence diagram of every element of the clean data. 

6. iterative_reconstruct_persist iterates over the previously written function reconstruct_persist by reformating the persistence diagrams of the clean data in a way that R will accept.

7. saveAllPersistenceForR takes in the output of iterative_reconstruct_persist and simply stores all of the persistence diagrams into CSV's. This is so that we can then load these CSVs into R and go right ahead in computing clusters. Be sure you are aware of what your current working directory is if you plan on running this function. 

In [14]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import loadtxt
import os
import shutil
import pandas as pd
import cython as cy

import dionysus as d
from dionysus import *

from sklearn.neighbors import NearestNeighbors

from ripser import ripser, plot_dgms
from ripser import Rips

import wfdb

r = Rips(maxdim = 2)

Rips(maxdim=2, thresh=inf, coeff=2, do_cocycles=False, verbose=True)


In [13]:
reference = pd.read_csv("REFERENCE-v3.csv", names = ["ECG run", "Type"])

N = [] #Store your data.
A = []
R = []
O = []
l = len(reference["ECG run"])

def kSublists(k): #Computes 4 organized lists of length k. 
    for i in range(l):    
        if reference["Type"][i] == 'N':
            if len(N) < k:
                N.append(reference["ECG run"][i])
        elif reference["Type"][i] == 'A':
            if len(A) < k:
                A.append(reference["ECG run"][i])
        elif reference["Type"][i] == '~':
            if len(R) < k:
                R.append(reference["ECG run"][i])
        elif reference["Type"][i] == "O":
            if len(O) < k:
                O.append(reference["ECG run"][i])

In [11]:
def reconstruct_persist(persistence): #Takes in an array of length i containing nx2 arrays, where n is the number
    total_dim = len(persistence)      #of points in the i-th dimension. The first column of the nx2 arrays contain
    length = 0                        #birth points, while the second column contains corresponding death points.
    k = 0 #keeps track of columns, see the for loop below
    for i in range(total_dim):
        length += len(persistence[i])
    persistence_Matrix = np.zeros(length*3).reshape(length, 3) #populate empty matrix
    
    for i in range(total_dim):
        k = 0
        if i != 0: #This if statement guarantees that the columns are stored correctly.
            for p in range(i):
                k += len(persistence[p])
        for j in range(len(persistence[i])):
            persistence_Matrix[j+k,0] = i #stores dimension in the first column
            persistence_Matrix[j+k,1] = persistence[i][j][0] #stores x coordinate, or birth
            persistence_Matrix[j+k,2] = persistence[i][j][1] #stores y coordinate, or death.
    return persistence_Matrix

def Eight_Hundred_Persistence_Test(folder): #Computes 800 persistence diagrams, given a directory of the location of 
    for i in range(800):                    #the data points. 
        if 0 <= i < 200:
            signals, fields = wfdb.rdsamp(folder + '/200 normal/' + N[i])
            diagram = r.fit_transform(signals)
            df = pd.DataFrame(reconstruct_persist(diagram))
            df.to_csv("Persistence Matrices/200 Normal Ps/" + N[i] + "PersMatrix.csv")
        if 200 <= i < 400:
            signals, fields = wfdb.rdsamp(folder + '/200 af/' + AF[i-200])
            diagram = r.fit_transform(signals)
            df = pd.DataFrame(reconstruct_persist(diagram))
            df.to_csv("Persistence Matrices/200 AF Ps/" + AF[i-200] + "PersMatrix.csv")            
        if 400 <= i < 600:
            signals, fields = wfdb.rdsamp(folder + '/200 random/' + O[i-400])
            diagram = r.fit_transform(signals)
            df = pd.DataFrame(reconstruct_persiste(diagram))
            df.to_csv("Persistence Matrices/200 Random Ps/" + O[i-400] + "PersMatrix.csv")            
        if 600 <= i < 800:
            signals, fields = wfdb.rdsamp(folder + '/200 other/' + R[i-600])
            diagram = r.fit_transform(signals)
            df = pd.DataFrame(reconstruct_persiste(diagram))
            df.to_csv("Persistence Matrices/200 Normal/" + R[i-600] + "PersMatrix.csv")        
  

In [12]:
N_clean = [] #Lists to store our clean data.
AF_clean = []
R_clean = []
O_clean = []

N_clean_labels = [] #Lists to keep track of the labels of our clean data.
AF_clean_labels = []
R_clean_labels = []
O_clean_labels = []

def TXT_to_ARRAY(direct): #Specify your GITHUB directory, i.e., wherever the AF, Normal, and Random folders are stored
    for file in os.listdir(direct + "/Normal"):
        filename = os.fsdecode(file)
        if len(N_clean) < 200:
            if filename.endswith(".txt"): #Hopefully you don't have any other .txt files in there...
                lines = np.array(pd.read_csv("TDA-MSRI-2018/Normal/" + filename))
                if np.shape(lines) != (): #We don't want empty empty vectors.
                    lines.reshape(len(lines), 1) #Reformats into a column vector.
                    if len(lines) > 1:    #Required double check so that we don't get single elements. 
                        N_clean.append(lines) 
                        N_clean_labels.append(filename)
            
    for file in os.listdir(direct + "/AF"):
        filename = os.fsdecode(file)
        if len(AF_clean) < 200:
            if filename.endswith(".txt"): 
                lines = np.array(pd.read_csv("TDA-MSRI-2018/AF/" + filename))
                if np.shape(lines) != ():
                    lines.reshape(len(lines), 1) 
                    if len(lines) > 1:
                        AF_clean.append(lines)
                        AF_clean_labels.append(filename)
            
    for file in os.listdir(direct + "/Random"):
        filename = os.fsdecode(file)
        if len(R_clean) < 200:
            if filename.endswith(".txt"): 
                lines = np.array(pd.read_csv("TDA-MSRI-2018/Random/" + filename))
                if np.shape(lines) != ():    
                    lines.reshape(len(lines), 1) 
                    if len(lines) > 1:    
                        R_clean.append(lines) 
                        R_clean_labels.append(filename)    

In [10]:
List_of_All = N_clean + AF_clean + R_clean

def Persistence_Lists(All_clean_vectors): #Takes in the clean vectors (noise removed) and outputs a list where each
    List_of_Persistence = []              #element is a clean vector's persistence diagram.
    for i in range(len(All_clean_vectors)):
        diagram = r.fit_transform(All_clean_vectors[i])
        List_of_Persistence.append(diagram)
    return List_of_Persistence

List_of_R_Persistence = []
def iterative_reconstruct_persist(thepersistence): #This function reformats the persistence diagrams into a form that 
    for i in range(len(All_clean_vectors)):                           #R can accept. This simply calls reconstruct_persist.
        List_of_R_Persistence.append(reconstruct_persist(thepersistence[i]))
        
Labels = R_clean_labels + N_clean_labels + AF_clean_labels
def saveAllPersistenceForR(final_list, theLabels): #final_list is the list of persistence diagrams written in a form 
    for i in range(600):                           #that R can accept. You plug in the output of iterative_reconstruct_persist.
        K = pd.DataFrame(final_list[i])
        K.to_csv(theLabels[i] + ".csv")